# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto (<font color="red">the first stage</font>)

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### Get the data from Wiki

In [2]:
html_resp = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(html_resp.content, 'html.parser')

### Get post codes table

In [3]:
tables = soup.findAll('table')
table = None
for t in tables:
    th_list = set(th.text.rstrip().upper() for th in t.findAll('th'))
    if len(th_list & {'POSTCODE', 'NEIGHBOURHOOD', 'BOROUGH' }) == 3:
        table = t
        break;
if table is None:
    raise RuntimeError("Can not obtain table!")

### Convert to DataFrame. Clean 'Not Assigned', group data etc (use x.upper() is not necessary in this case, but sometimes  it's a good practice)

In [4]:
post_codes_df = pd.read_html(str(table))[0]
# rename columns
post_codes_df.columns = ['PostalCode','Borough','Neighborhood']
# remove empty rows
post_codes_df = post_codes_df[post_codes_df['Borough'].apply(lambda x: x.upper()) != 'NOT ASSIGNED']
# assign 'Borough' to 'Neighborhood' where the value is 'Not assigned'
post_codes_df['Neighborhood'] = np.where(post_codes_df['Neighborhood'].apply(lambda x: x.upper()) ==  'NOT ASSIGNED',
    post_codes_df['Borough'],
    post_codes_df['Neighborhood'])
# group by 'PostalCode' 
post_codes_df = post_codes_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).to_frame().reset_index()

### Print Shape (total rows)

In [5]:
post_codes_df.shape[0]

103